In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def get_date_status():
    df = pd.read_excel('holiday_94_to_98.xlsx');
    date = dict()
    holiday = dict()
    for row in np.array(df):
        p_date = row[0].replace('-' , '/')
        g_date = datetime.strptime(row[1], '%Y-%m-%d')
        hday   = 0
        if row[2] :
            hday = 1
        #Monday is 0 and Sunday is 6 by function weekday() 
        date[p_date] = g_date.weekday()
        holiday[p_date] = hday
    return holiday , date

In [3]:
holiday_dict, week_dict = get_date_status()

In [4]:
dff = pd.ExcelFile('train_without_sum.xlsx')

In [5]:
sheets = dff.sheet_names[20:]

In [6]:
for st in sheets:
    print('sh')
    df = pd.read_excel('train_without_sum.xlsx', sheetname=st, header=None, names=["Date", "Time", "Energy"])
    date = np.array([ i.split('/')  for i in np.array(df['Date']) ])
    time = np.array([ str(i).split(' ')[-1].split(':')  for i in np.array(df['Time']) ])
    df['Year'] = [ float(i) for i in date[:,0]]
    df['Month'] = [ float(i) for i in date[:,1]]
    df['Day'] = [ float(i) for i in date[:,2]]
    df['Week'] = [ week_dict[i] for i in np.array(df['Date'])]


    df['Hour'] = [ float(i) for i in time[:,0]]
    df['Minute'] = [ float(i) for i in time[:,1]]

    df['Holiday'] = [ holiday_dict[i] for i in np.array(df['Date'])]
    df['Hour_sin'] = np.sin(df['Hour']*(2.*np.pi/24))
    df['Hour_cos'] = np.cos(df['Hour']*(2.*np.pi/24))

    df['Month_sin'] = np.sin((df['Month']-1)*(2.*np.pi/12))
    df['Month_cos'] = np.cos((df['Month']-1)*(2.*np.pi/12))

    df['Day_sin'] = np.sin((df['Day']-1)*(2.*np.pi/31))
    df['Day_cos'] = np.cos((df['Day']-1)*(2.*np.pi/31))


    df['Week_sin'] = np.sin((df['Week'])*(2.*np.pi/7))
    df['Week_cos'] = np.cos((df['Week'])*(2.*np.pi/7))

    df['Value'] = df['Energy']

    df = df.filter(['Year' , 'Month_sin', 'Month_cos' , 'Day_sin', 'Day_cos' , 'Week_sin' , 'Week_cos' , 'Hour_sin', 'Hour_cos' , 'Holiday', 'Value'])

    mean = df['Value'].mean(axis=0)
    std = df['Value'].std(axis=0)
    print("mean : " , mean, " , std: ", std)
    df2 = df.loc[ abs (df['Value'] - mean) < 2.5 * std ]
    print('removed ' , df.shape[0] - df2.shape[0] , 'data ')
    df = df2
    columns = df.columns[0:-1]
    for col in columns:
        mnCol = float(df[col].min())
        mxCol = float(df[col].max())
        dur = mxCol - mnCol
        if dur == 0:
            dur = 1
        df[col] = [ ((float(u) - mnCol) / dur) for u in np.array(df[col]) ]
    df.to_excel(r'prepared_train_data' + st + '.xlsx', index=False, header =True)

/usr/local/lib/python3.5/dist-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


AttributeError: 'numpy.float64' object has no attribute 'split'